# Quantum Walks on the Metatron Cube

This notebook introduces quantum walks on the Metatron Cube using the `metatron_qso` Python SDK.

## What is a Quantum Walk?

A quantum walk is the quantum analog of a classical random walk. Instead of probabilistic transitions, quantum walks use quantum superposition and interference, leading to fundamentally different spreading behavior.

## The Metatron Cube

The Metatron Cube is a 13-node graph with 78 edges representing sacred geometry:
- **13 nodes**: 1 center + 6 hexagon vertices + 6 cube vertices
- **78 edges**: Highly connected structure
- **Symmetry**: Contains all five Platonic solids

In [ ]:
# Import the Metatron QSO library
import metatron_qso
import matplotlib.pyplot as plt
import numpy as np

print(f"Metatron QSO version: {metatron_qso.__version__}")

## 1. Create the Metatron Cube Graph

In [ ]:
# Create the graph
graph = metatron_qso.MetatronGraph()

print(graph)
print(f"Number of nodes: {graph.num_nodes()}")
print(f"Number of edges: {graph.num_edges()}")

## 2. Run a Quantum Walk

We'll start with the quantum state localized at the central node (node 0) and evolve it over time.

In [ ]:
# Run quantum walk from central node
result = metatron_qso.run_quantum_walk(
    graph=graph,
    source_nodes=[0],  # Start at central node
    t_max=10.0,        # Evolution time
    dt=0.05            # Time step
)

times = result['times']
probabilities = result['probabilities']

print(f"Computed {len(times)} time steps")
print(f"Time range: {times[0]:.2f} to {times[-1]:.2f}")

## 3. Visualize the Probability Evolution

Let's plot how the probability distribution evolves over time.

In [ ]:
# Convert to numpy array for easier plotting
prob_array = np.array(probabilities)

# Plot evolution of probability at each node
plt.figure(figsize=(12, 6))

# Plot for first few nodes
nodes_to_plot = [0, 1, 2, 3, 7, 8]
for node in nodes_to_plot:
    plt.plot(times, prob_array[:, node], label=f'Node {node}', linewidth=2)

plt.xlabel('Time', fontsize=12)
plt.ylabel('Probability', fontsize=12)
plt.title('Quantum Walk Probability Evolution on Metatron Cube', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Analyze Quantum Interference

Quantum walks exhibit characteristic oscillations due to quantum interference. Let's analyze the central node probability.

In [ ]:
# Focus on central node (node 0)
central_prob = prob_array[:, 0]

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(times, central_prob, linewidth=2, color='blue')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Probability', fontsize=12)
plt.title('Central Node Probability', fontsize=14)
plt.grid(True, alpha=0.3)

# Compute total spreading (entropy)
def entropy(probs):
    """Compute Shannon entropy of probability distribution"""
    p = np.array(probs)
    p = p[p > 0]  # Remove zeros
    return -np.sum(p * np.log(p))

entropies = [entropy(p) for p in probabilities]

plt.subplot(1, 2, 2)
plt.plot(times, entropies, linewidth=2, color='red')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Entropy', fontsize=12)
plt.title('State Entropy (Spreading)', fontsize=14)
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Initial entropy: {entropies[0]:.4f}")
print(f"Final entropy: {entropies[-1]:.4f}")
print(f"Max possible entropy: {np.log(graph.num_nodes()):.4f}")

## 5. Compare Different Starting Configurations

Let's see how the walk behaves when starting from different nodes.

In [ ]:
# Compare walks from different starting nodes
start_nodes_configs = [
    ([0], 'Central node'),
    ([1], 'Hexagon vertex'),
    ([7], 'Cube vertex'),
    ([1, 2, 3], 'Multiple hexagon vertices')
]

plt.figure(figsize=(14, 8))

for idx, (start_nodes, label) in enumerate(start_nodes_configs, 1):
    result = metatron_qso.run_quantum_walk(
        graph=graph,
        source_nodes=start_nodes,
        t_max=8.0,
        dt=0.1
    )
    
    times = result['times']
    entropies = [entropy(p) for p in result['probabilities']]
    
    plt.plot(times, entropies, linewidth=2, label=label)

plt.xlabel('Time', fontsize=12)
plt.ylabel('Entropy', fontsize=12)
plt.title('Quantum Walk Spreading from Different Initial States', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Snapshot Visualization

Let's visualize the probability distribution at specific time points.

In [ ]:
# Run a fresh walk
result = metatron_qso.run_quantum_walk(
    graph=graph,
    source_nodes=[0],
    t_max=6.0,
    dt=0.1
)

# Select time snapshots
snapshot_times = [0.0, 1.0, 2.0, 4.0]
times = result['times']
probabilities = result['probabilities']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for ax, t_target in zip(axes, snapshot_times):
    # Find closest time index
    idx = min(range(len(times)), key=lambda i: abs(times[i] - t_target))
    t_actual = times[idx]
    probs = probabilities[idx]
    
    # Bar plot of probabilities
    nodes = list(range(len(probs)))
    colors = ['red' if i == 0 else 'blue' for i in nodes]
    
    ax.bar(nodes, probs, color=colors, alpha=0.7)
    ax.set_xlabel('Node', fontsize=11)
    ax.set_ylabel('Probability', fontsize=11)
    ax.set_title(f't = {t_actual:.2f}', fontsize=12)
    ax.set_ylim([0, 1.0])
    ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Conclusion

In this notebook, we've explored:
1. Creating a Metatron Cube graph
2. Running quantum walks with different initial conditions
3. Visualizing probability evolution and quantum interference
4. Analyzing state spreading using entropy

Quantum walks on the Metatron Cube show rich dynamics due to:
- **Quantum interference**: Oscillations in probability
- **Graph symmetry**: Uniform spreading patterns
- **High connectivity**: Fast probability distribution

## Next Steps

- Try different evolution times and time steps
- Experiment with multi-node initial states
- Explore QAOA and VQE algorithms (see other examples)
- Analyze spectral properties of the Hamiltonian